In [ ]:
from concurrent import futures
import grpc
from pkg.proto.aiservice.v1 import aiservice_pb2
from pkg.proto.aiservice.v1 import aiservice_pb2_grpc
from pkg.proto.aiservice.v1.aiservice_pb2 import PromptContentRequest, GenerateTitleAndTagResponse
from internal.pkg.runnable.generateTitleAndTagChain import GenerateTitleAndTagChain, OutputModel


import asyncio

class AIService(aiservice_pb2_grpc.AIService):
    async def GenerateTitleAndTag(self, request:aiservice_pb2.PromptContentRequest, context:grpc.ServicerContext):
        print("requested")
        resp:OutputModel = await GenerateTitleAndTagChain.ainvoke(request.prompt, request.content)
        return GenerateTitleAndTagResponse(title=resp.title, tags=resp.tags)

async def serve():
    #service中的方法需要是线程安全的。
    server = grpc.aio.server(futures.ThreadPoolExecutor(max_workers=10))
    aiservice_pb2_grpc.add_AIServiceServicer_to_server(AIService(), server)
    server.add_insecure_port('[::]:50051')
    await server.start()
    print("Server started, listening on port 50051")
    await server.wait_for_termination()

import nest_asyncio  # pip install nest_asyncio
nest_asyncio.apply()  # 允许嵌套事件循环

if __name__ == '__main__':
    await serve()
    # asyncio.run(serve())

Server started, listening on port 50051
requested
requested
requested
